# KD only for ViT Tiny Student and ViT Small Teacher

In [ ]:
import os
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.transforms import AutoAugment, AutoAugmentPolicy, RandomErasing
from transformers import (
    DeiTConfig,
    DeiTForImageClassification,
    ViTConfig,
    ViTPreTrainedModel,
    ViTModel
)
from tqdm import tqdm

# ---------------------------------------------
# Setup
# ---------------------------------------------
use_dp = True  # Use DataParallel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
np.random.seed(42)

mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)

alpha, temp = 0.5, 4.0
resolution = 32
epochs = 60
batch_size = 128

# ---------------------------------------------
# ViT with Distillation
# ---------------------------------------------
class ViTWithDistillation(ViTPreTrainedModel):
    def __init__(self, config: ViTConfig):
        super().__init__(config)
        self.vit = ViTModel(config)
        self.distill_token = nn.Parameter(torch.zeros(1, 1, config.hidden_size))
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.distiller = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()

    def forward(self, pixel_values, labels=None, teacher_logits=None,
                alpha=0.5, temperature=1.0):
        B = pixel_values.size(0)
        embeds = self.vit.embeddings(pixel_values, interpolate_pos_encoding=True)
        cls_emb, patch_emb = embeds[:, :1, :], embeds[:, 1:, :]
        dist_tok = self.distill_token.expand(B, -1, -1)
        x = torch.cat([cls_emb, dist_tok, patch_emb], dim=1)
        x = self.vit.encoder(x)[0]
        cls_out, dist_out = x[:, 0], x[:, 1]
        logits = self.classifier(cls_out)
        dist_logits = self.distiller(dist_out)
        output = {"logits": logits, "distill_logits": dist_logits}

        if labels is not None and teacher_logits is not None:
            loss_ce = F.cross_entropy(logits, labels)
            kd = F.kl_div(
                F.log_softmax(dist_logits / temperature, dim=1),
                F.softmax(teacher_logits / temperature, dim=1),
                reduction='batchmean'
            ) * (temperature ** 2)
            output["loss"] = (1 - alpha) * loss_ce + alpha * kd

        return output

# ---------------------------------------------
# Prepare Teacher
# ---------------------------------------------
teacher_config = DeiTConfig(
    image_size=resolution,
    patch_size=2,
    num_labels=10,
    hidden_size=384,
    num_hidden_layers=12,
    num_attention_heads=6,
    intermediate_size=1536,
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    stochastic_depth_prob=0.1
)

teacher = DeiTForImageClassification(teacher_config).to(device)
if use_dp:
    teacher = nn.DataParallel(teacher)

ckpt = torch.load("/kaggle/input/best-teacher/pytorch/default/1/best_teacher.pth", map_location=device)
# Remove "module." if present in keys
state = {k.replace("module.", ""): v for k, v in ckpt.items()}
teacher.module.load_state_dict(state, strict=True) if use_dp else teacher.load_state_dict(state, strict=True)
teacher.eval()
for p in teacher.parameters():
    p.requires_grad_(False)

# ---------------------------------------------
# Data Preparation
# ---------------------------------------------
train_tf = transforms.Compose([
    transforms.RandomCrop(resolution, padding=4),
    transforms.RandomHorizontalFlip(),
    AutoAugment(policy=AutoAugmentPolicy.CIFAR10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    RandomErasing(p=0.2, scale=(0.02, 0.2), ratio=(0.3, 3.3))
])

val_tf = transforms.Compose([
    transforms.CenterCrop(resolution),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

dataset = datasets.CIFAR10('./data', train=True, download=True, transform=train_tf)
num_val = int(0.1 * len(dataset))
train_idx, val_idx = torch.utils.data.random_split(
    list(range(len(dataset))), [len(dataset) - num_val, num_val],
    generator=torch.Generator().manual_seed(42)
)
train_ds = Subset(dataset, train_idx)
val_ds = Subset(datasets.CIFAR10('./data', train=True, download=False, transform=val_tf), val_idx)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=4)

# ---------------------------------------------
# Initialize Student
# ---------------------------------------------
stu_cfg = ViTConfig(
    image_size=resolution,
    patch_size=2,
    num_labels=10,
    hidden_size=192,
    num_hidden_layers=12,
    num_attention_heads=3,
    intermediate_size=768,
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    stochastic_depth_prob=0.1
)

student = ViTWithDistillation(config=stu_cfg).to(device)
if use_dp:
    student = nn.DataParallel(student)

opt = optim.AdamW(student.parameters(), lr=3e-4, weight_decay=1e-4)

# ---------------------------------------------
# Training Loop
# ---------------------------------------------
for ep in range(1, epochs + 1):
    student.train()
    loop = tqdm(train_loader, desc=f"Train Epoch {ep}/{epochs}")
    for x, y in loop:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            tlog = teacher(pixel_values=x, interpolate_pos_encoding=True).logits
        out = student(pixel_values=x, labels=y, teacher_logits=tlog,
                      alpha=alpha, temperature=temp)
        loss = out['loss'].mean()
        opt.zero_grad()
        loss.backward()
        opt.step()
        loop.set_postfix(loss=loss.item())

    # Validation
    student.eval()
    correct, total = 0, 0
    val_loop = tqdm(val_loader, desc=f"Val Epoch {ep}/{epochs}")
    for xb, yb in val_loop:
        xb, yb = xb.to(device), yb.to(device)
        with torch.no_grad():
            preds = student(pixel_values=xb)['logits'].argmax(1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)
    val_acc = 100 * correct / total
    print(f"Epoch {ep} Validation Accuracy: {val_acc:.2f}%")

# ---------------------------------------------
# Final Test
# ---------------------------------------------
student.eval()
test_ds = datasets.CIFAR10('./data', train=False, transform=val_tf)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4)
correct = 0
for xb, yb in tqdm(test_loader, desc="Final Test"):
    xb, yb = xb.to(device), yb.to(device)
    with torch.no_grad():
        preds = student(pixel_values=xb)['logits'].argmax(1)
    correct += (preds == yb).sum().item()

student_acc = 100 * correct / len(test_ds)
print(f"Final Test Accuracy: {student_acc:.2f}%")

2025-05-06 19:02:56.190837: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746558176.393674      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746558176.455182      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_31/2935911582.py:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_onl

100%|██████████| 170M/170M [00:02<00:00, 77.2MB/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data


Train Epoch 1/60:   0%|          | 0/352 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Val Epoch 1/60: 100%|██████████| 40/40 [00:05<00:00,  7.44it/s]


Epoch 1 Validation Accuracy: 32.60%


...

Val Epoch 25/60: 100%|██████████| 40/40 [00:04<00:00,  8.39it/s]


Epoch 25 Validation Accuracy: 75.24%


Val Epoch 26/60: 100%|██████████| 40/40 [00:04<00:00,  8.34it/s]


Epoch 26 Validation Accuracy: 74.30%


Train Epoch 27/60:  38%|███▊      | 133/352 [01:28<02:25,  1.51it/s, loss=0.669]

Training for the remaining epochs, Kaggle crashed.

This runs for the remaining 33 epochs

In [2]:
# ---------------------------------------------
# Training Loop
# ---------------------------------------------
for ep in range(1, 33 + 1):
    student.train()
    loop = tqdm(train_loader, desc=f"Train Epoch {ep}/{epochs}")
    for x, y in loop:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            tlog = teacher(pixel_values=x, interpolate_pos_encoding=True).logits
        out = student(pixel_values=x, labels=y, teacher_logits=tlog,
                      alpha=alpha, temperature=temp)
        loss = out['loss'].mean()
        opt.zero_grad()
        loss.backward()
        opt.step()
        loop.set_postfix(loss=loss.item())

    # Validation
    student.eval()
    correct, total = 0, 0
    val_loop = tqdm(val_loader, desc=f"Val Epoch {ep}/{epochs}")
    for xb, yb in val_loop:
        xb, yb = xb.to(device), yb.to(device)
        with torch.no_grad():
            preds = student(pixel_values=xb)['logits'].argmax(1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)
    val_acc = 100 * correct / total
    print(f"Epoch {ep} Validation Accuracy: {val_acc:.2f}%")

# ---------------------------------------------
# Final Test
# ---------------------------------------------
student.eval()
test_ds = datasets.CIFAR10('./data', train=False, transform=val_tf)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=4)
correct = 0
for xb, yb in tqdm(test_loader, desc="Final Test"):
    xb, yb = xb.to(device), yb.to(device)
    with torch.no_grad():
        preds = student(pixel_values=xb)['logits'].argmax(1)
    correct += (preds == yb).sum().item()

student_acc = 100 * correct / len(test_ds)
print(f"Final Test Accuracy: {student_acc:.2f}%")

Val Epoch 1/60: 100%|██████████| 40/40 [00:04<00:00,  8.40it/s]


Epoch 1 Validation Accuracy: 76.66%


Val Epoch 2/60: 100%|██████████| 40/40 [00:04<00:00,  8.41it/s]


Epoch 2 Validation Accuracy: 74.78%


Val Epoch 3/60: 100%|██████████| 40/40 [00:04<00:00,  8.31it/s]


...

Epoch 32 Validation Accuracy: 81.46%


Val Epoch 33/60: 100%|██████████| 40/40 [00:04<00:00,  8.26it/s]


Epoch 33 Validation Accuracy: 82.12%


Final Test: 100%|██████████| 79/79 [00:09<00:00,  8.41it/s]

Final Test Accuracy: 82.28%
